<a href="https://colab.research.google.com/github/shusank8/DiffusionModels/blob/main/VisualizingLatentSpace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Visualizing Auto Encoders hello world")

Visualizing Auto Encoders hello world


In [2]:
import torch
import torch.nn as nn

In [3]:
import torchvision
import torchvision.datasets as datasets
from torchvision.transforms import v2
from torch.utils.data import DataLoader

In [4]:
transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale =True),
    # v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms)


In [6]:
train_dataloader = DataLoader(mnist_trainset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(mnist_testset, batch_size=64, shuffle=True)

In [7]:
len(train_dataloader), len(test_dataloader)

(938, 157)

In [8]:
for x in train_dataloader:
  print(x[0].min(),x[0].max())
  break

tensor(0.) tensor(1.)


In [24]:
class VisualizingLatentSpace(nn.Module):

  def __init__(self, latentdim):
    super().__init__()
    self.latentdim = latentdim
    self.enoder = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride = 1, padding=1),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5, stride = 1),
        nn.AvgPool2d(kernel_size=2),
        nn.ReLU(),

        nn.Conv2d(in_channels=16, out_channels=8, kernel_size=5, stride = 1),
        nn.Conv2d(in_channels=8, out_channels=1, kernel_size=4, stride = 1),
        nn.AvgPool2d(kernel_size=(4,2)),
        nn.Sigmoid(),
    )

    self.decoder = nn.Sequential(
        nn.ConvTranspose2d(in_channels=1, out_channels=3, kernel_size=(3,2)),
        nn.ConvTranspose2d(in_channels=3, out_channels=8, kernel_size=4,stride = 2),
        nn.ReLU(),
        nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=4,stride = 2),
        nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=4),
        nn.ReLU(),
        nn.ConvTranspose2d(in_channels=8, out_channels=3, kernel_size=5),
        nn.ConvTranspose2d(in_channels=3, out_channels=1, kernel_size=4),
        nn.Sigmoid(),
    )

  def forward(self, x):
    enc = self.enoder(x)
    dec = self.decoder(enc)
    return [enc, dec]



In [29]:
# Create a generator
gen = torch.Generator()

# Set the seed for reproducibility
gen.manual_seed(42)

dim = 2
model = VisualizingLatentSpace(dim)
for param in model.parameters():
  torch.nn.init.normal_(param, mean=0.0, std=0.8, generator=gen)
lossfn = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

In [30]:
vis = []
for i in range(5):
  for x in train_dataloader:

    model = VisualizingLatentSpace(2)
    for img, label in train_dataloader:
      enc, out = model(img)
      loss = lossfn(out, img)
      optimizer.zero_grad(set_to_none=True)
      loss.backward()
      optimizer.step()
    vis.append((i, enc, label))
      # print(loss.item())

KeyboardInterrupt: 